In [2]:
# 2_ModelTraining.ipynb

# --------------------------------------------
# 1. IMPORT LIBRARIES
# --------------------------------------------
import numpy as np
import pandas as pd
import warnings
import joblib  # for saving/loading scaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Suppress protobuf warnings from TensorFlow
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

# --------------------------------------------
# 2. LOAD DATA FROM PREVIOUS NOTEBOOK
# --------------------------------------------

X_train = np.load('../data/X_train.npy')
X_test = np.load('../data/X_test.npy')
y_train = np.load('../data/y_train.npy')
y_test = np.load('../data/y_test.npy')

# Load the scaler used during training
scaler = joblib.load('../models/scaler.save')

# --------------------------------------------
# 3. BUILD MODEL (Keras ANN)
# --------------------------------------------

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --------------------------------------------
# 4. TRAIN MODEL
# --------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)

# --------------------------------------------
# 5. EVALUATE MODEL
# --------------------------------------------

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_prob))

# --------------------------------------------
# 6. SAVE MODEL
# --------------------------------------------

model.save('../models/churn_model.h5')

# --------------------------------------------
# 7. EXPORT PREDICTIONS (Optional)
# --------------------------------------------

# Load original dataset
df = pd.read_csv('../data/BankChurners.csv')

# Drop columns (ignore errors if missing)
drop_cols = [
    'CLIENTNUM',
    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon',
    'Naive_Bayes_Classifier_Attrition_Flag_Income_Category_Age'
]
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Create target column and drop original
df['Churn'] = df['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)
df.drop('Attrition_Flag', axis=1, inplace=True)

# One-hot encode with the same columns as training:
df_encoded = pd.get_dummies(df, drop_first=True)

# Make sure columns in df_encoded match training features exactly
# Load training feature columns list
train_features = joblib.load('../models/train_features.save')

# Reindex to training columns, fill missing cols with 0
df_encoded = df_encoded.reindex(columns=train_features, fill_value=0)

# Scale features with loaded scaler
X_full_scaled = scaler.transform(df_encoded)

# Predict churn probabilities
probs_full = model.predict(X_full_scaled)

df['Churn_Prob'] = probs_full

# Save churn probabilities to CSV
df[['Churn_Prob']].to_csv('../data/churn_probabilities.csv', index=False)

print("\nChurn probabilities saved to '../data/churn_probabilities.csv'")


Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


203/203 ━━━━━━━━━━━━━━━━━━━━ 1s 902us/step - accuracy: 0.9272 - loss: 0.2039 - val_accuracy: 0.9988 - val_loss: 0.0120
Epoch 2/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - accuracy: 0.9985 - loss: 0.0155 - val_accuracy: 1.0000 - val_loss: 7.7932e-04
Epoch 3/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.9995 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 2.3768e-04
Epoch 4/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.9997 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 8.8475e-05
Epoch 5/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.9997 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 7.1950e-05
Epoch 6/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 1.0000 - loss: 8.6087e-04 - val_accuracy: 1.0000 - val_loss: 4.2704e-05
Epoch 7/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.9998 - loss: 7.0227e-04 - val_accuracy: 1.0000 - val_loss: 2.0852e-05
Epoch 8/100
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accura


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1699
           1       1.00      1.00      1.00       327

    accuracy                           1.00      2026
   macro avg       1.00      1.00      1.00      2026
weighted avg       1.00      1.00      1.00      2026

ROC AUC Score: 1.0
317/317 ━━━━━━━━━━━━━━━━━━━━ 0s 213us/step

Churn probabilities saved to '../data/churn_probabilities.csv'
